Segmenting and Clustering Neighborhoods in Toronto, Scrape Data Assignment 1

#Import needed installs

In [10]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests

#Import libraries

In [11]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

#Pull data from url

In [12]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
content = BeautifulSoup(requests.get(url).content, 'lxml')

#Turn data into table

In [13]:
table = content.find('table')
td = table.find_all('td')
postcode = []
borough = []
neighborhood = []

for i in range(0, len(td), 3):
    postcode.append(td[i].text.strip())
    borough.append(td[i+1].text.strip())
    neighborhood.append(td[i+2].text.strip())
    
#Transform dataframe into three columns
wiki_df = pd.DataFrame(data=[postcode, borough, neighborhood]).transpose()
wiki_df.columns = ['Postal Code', 'Borough', 'Neighborhood']

#Ditch cells with borough that are not assigned
wiki_df['Borough'].replace('Not assigned', np.nan, inplace=True)
wiki_df.dropna(subset=['Borough'], inplace=True)

#Replace cells in borough that do not have an assigned neighborhood
wiki_df['Neighborhood'].replace('Not assigned', "Queen's Park", inplace=True)
                                
#Combine rows for simplicity
wiki_df = wiki_df.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
wiki_df.columns = ['Postal Code', 'Borough', 'Neighborhood']
wiki_df.head(12)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West
9,M1N,Scarborough,Birch Cliff / Cliffside West


In [14]:
wiki_df.shape

(103, 3)